<a href="https://colab.research.google.com/github/cserock/colab-examples/blob/main/07_RAG_pdf_%EC%98%88%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 환경변수 파일(.env) 설정하기
/content/drive/MyDrive/lg-dx/에 아래 내용으로 .env 파일을 작성한 후 업로드합니다.  

OPENAI_API_KEY=sk-xxxx  
LANGCHAIN_TRACING_V2="true"  
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"  
LANGCHAIN_API_KEY="lsv2_xxxx  
LANGCHAIN_PROJECT="lg-dx"


Google Drive 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

python-dotenv 라이브러리 설치

In [ ]:
!pip install python-dotenv

환경변수 파일 로드 및 확인

In [ ]:
from dotenv import load_dotenv
load_dotenv('/content/drive/MyDrive/lg-dx/.env', override=True)

import os
openai_api_key = os.environ.get('OPENAI_API_KEY')
print("openai_api_key : " + openai_api_key)
langchain_api_key = os.environ.get('LANGCHAIN_API_KEY')
print("langchain_api_key : " + langchain_api_key)

## LangChain 설치 및 업데이트

In [ ]:
# LangChain 설치 및 업데이트
!pip install -qU langchain-teddynote langchain_community pymupdf faiss-cpu

설치된 LangChain 버전을 확인합니다.

In [ ]:
from importlib.metadata import version

print("[LangChain 관련 패키지 버전]")
for package_name in [
    "langchain",
    "langchain-core",
    "langchain-experimental",
    "langchain-community",
    "langchain-openai",
    "langchain-teddynote",
    "langchain-huggingface",
    "langchain-google-genai",
    "langchain-anthropic",
    "langchain-cohere",
    "langchain-chroma",
    "langchain-elasticsearch",
    "langchain-upstage",
    "langchain-cohere",
    "langchain-milvus",
    "langchain-text-splitters",
]:
    try:
        package_version = version(package_name)
        print(f"{package_name}: {package_version}")
    except ImportError:
        print(f"{package_name}: 설치되지 않음")

LangSmith 추적 설정  
.env 파일에 LangSmith API 키가 설정 되어 있어야 합니다.(LANGCHAIN_API_KEY)

In [ ]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("lg-dx-rag-pdf")

## PDF 파일 다운로드
실습을 위해 다운로드 받은 파일을 data 폴더로 복사해 주세요.

소프트웨어정책연구소(SPRi) - 2023년 12월호  

저자: 유재흥(AI정책연구실 책임연구원), 이지수(AI정책연구실 위촉연구원)  
링크: https://spri.kr/posts/view/23669  
파일명: SPRI_AI_Brief_2023년12월호_F.pdf  


LangSmith 추적 설정  
.env 파일에 LangSmith API 키가 설정 되어 있어야 합니다.(LANGCHAIN_API_KEY)

In [ ]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("lg-dx-rag")

아래와 같이 추적을 끌 수 있습니다.

In [ ]:
from langchain_teddynote import logging

# set_enable=False 로 지정하면 추적을 하지 않습니다.
logging.langsmith("lg-dx-rag", set_enable=False)

## RAG 기본 파이프라인(1~8단계)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [ ]:
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("/content/drive/MyDrive/lg-dx/data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

In [ ]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")

In [ ]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [ ]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
# 검색 결과 4개(default)
# retriever = vectorstore.as_retriever()

# 검색 결과 8개
retriever = vectorstore.as_retriever(search_kwargs={"k": 8})


In [ ]:
# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.
retriever.invoke("삼성전자가 자체 개발한 AI 의 이름은?")

In [ ]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Answer in Korean.

#Question:
{question}
#Context:
{context}

#Answer:"""
)

In [ ]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [ ]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "삼성전자가 자체 개발한 AI 의 이름은?"
response = chain.invoke(question)
print(response)